# CMIP6 predictions

In [15]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import random as rd
from lmfit import Model
from lmfit.model import save_modelresult, load_modelresult
from scipy import optimize
plt.rcParams.update({'figure.max_open_warning': 0})
import importlib
from data_loading_functions import *
import xarray as xr

In [ ]:
exp = 'abrupt-4xCO2'
#exp = 'abrupt-2xCO2'
#exp = 'abrupt-0p5xCO2'

model_anom_names = [ f.name for f in os.scandir('../../CMIP6-forcing/Processed_data/Global_annual_anomalies/') if f.is_dir() and f.name !='.ipynb_checkpoints']
model_anom_names.sort()

member_dict = {}
for model in model_anom_names:
    anom_exp = [ f.name for f in os.scandir('../../CMIP6-forcing/Processed_data/Global_annual_anomalies/' + model) if f.is_dir() and f.name !='.ipynb_checkpoints']
    if exp in anom_exp:
        member_dict[model] = find_members(model, exp)
member_dict

In [17]:
def exp_part1(t, S1, tau1):
    return S1*(1 - np.exp(-t/tau1))
def exp_part2(t, S2, tau2):
    return S2*(1 - np.exp(-t/tau2))
def exp_part3(t, S3, tau3):
    return S3*(1 - np.exp(-t/tau3))

def osc_part1(t, Sosc1, taup, Tq):
    p = -1/taup; q = 2*np.pi/Tq
    osc_part1 = Sosc1*(1 - np.exp(-t/taup)*(np.cos(q*t) + p/q*np.sin(q*t)))
    return osc_part1
def osc_part2(t, Sosc2, taup, Tq):
    p = -1/taup; q = 2*np.pi/Tq
    osc_part2 = Sosc2*(1 - np.exp(-t/taup)*(np.cos(q*t) - q/p*np.sin(q*t)))
    return osc_part2
def osc_parts(t, Sosc1, Sosc2, taup, Tq):
    p = -1/taup; q = 2*np.pi/Tq
    osc_part1 = Sosc1*(1 - np.exp(-t/taup)*(np.cos(q*t) + p/q*np.sin(q*t)))
    osc_part2 = Sosc2*(1 - np.exp(-t/taup)*(np.cos(q*t) - q/p*np.sin(q*t)))
    return osc_part1 + osc_part2

def twoexp_function(t, S1, S2, tau1, tau2):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2)
def threeexp_function(t, S1, S2, S3, tau1, tau2, tau3):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2) + exp_part3(t, S3, tau3)
def expandosc_function(t, S1, S2, Sosc1, Sosc2, tau1, tau2, taup, Tq):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2) + osc_parts(t, Sosc1, Sosc2, taup, Tq)

exp_model1 = Model(exp_part1); exp_model2 = Model(exp_part2); 
twoexp_model = Model(twoexp_function);
twoexp_model.set_param_hint('S1', value=4, min=0, max=10.0)
twoexp_model.set_param_hint('S2', value=2, min=0, max=10.0)
twoexp_model.set_param_hint('tau1', value=4, min=0, max=8)
twoexp_model.set_param_hint('tau2', value=20, min=8, max=1000)

exp_model3 = Model(exp_part3); threeexp_model = Model(threeexp_function);
threeexp_model.set_param_hint('S1', value=4, min=0, max=10.0)
threeexp_model.set_param_hint('S2', value=2, min=0, max=10.0)
threeexp_model.set_param_hint('S3', value=2, min=0, max=10.0)
threeexp_model.set_param_hint('tau1', value=4, min=0, max=8)
threeexp_model.set_param_hint('tau2', value=20, min=8, max=100)
threeexp_model.set_param_hint('tau3', value=200, min=100, max=1000)

osc_model = Model(osc_parts); expandosc_model = Model(expandosc_function)
expandosc_model.set_param_hint('S1', value=4, min=0, max=10.0)# guess the same as for two-box model 
expandosc_model.set_param_hint('S2', value=2, min=0, max=10.0)
expandosc_model.set_param_hint('Sosc1', value=0.5, min=0, max=4.0)
expandosc_model.set_param_hint('Sosc2', value=0.5, min=0, max=4.0)
expandosc_model.set_param_hint('tau1', value=4, min=0, max=8)
expandosc_model.set_param_hint('tau2', value=20, min=8, max=1000)


expandosc_model.set_param_hint('taup', value=600, min=20, max=1000)
#expandosc_model.set_param_hint('Tq', value=400, min=40, max=2000) # used for longrunmip
expandosc_model.set_param_hint('Tq', value=100, min=40, max=2000)

In [18]:
# define a forcing:
def linear_to140years(max_t = 150): # forcing that increases linearly to 1 for the first 140 years, thereafter it remains constant
    forcing_first140 = np.arange(0,141)/140 # including year 0 in array
    forcing_constant = np.tile(1, max_t-140)
    return np.concatenate([forcing_first140, forcing_constant])

In [19]:

def exp_part1_fixedpar(t, par_values):
    return exp_part1(t, S1 = par_values['S1'], tau1 = par_values['tau1'])
def exp_part2_fixedpar(t, par_values):
    return exp_part2(t, S2 = par_values['S2'], tau2 = par_values['tau2'])
def exp_part3_fixedpar(t, par_values):
    return exp_part3(t, S3 = par_values['S3'], tau3 = par_values['tau3'])    
    
def osc_parts_fixedpar(t, par_values):
    return osc_parts(t, Sosc1 = par_values['Sosc1'], Sosc2 = par_values['Sosc2'], taup = par_values['taup'], Tq = par_values['Tq'])
def osc_part1_fixedpar(t, par_values):
    return osc_part1(t, Sosc1 = par_values['Sosc1'], taup = par_values['taup'], Tq = par_values['Tq'])
def osc_part2_fixedpar(t, par_values):
    return osc_part2(t, Sosc2 = par_values['Sosc2'], taup = par_values['taup'], Tq = par_values['Tq'])

def twobox_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values)
def threebox_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values) + exp_part3_fixedpar(t, par_values)
def oscillatory_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values) + osc_parts_fixedpar(t, par_values)

## Plot all model responses to a linearly increasing forcing for 4-box model with oscillations

In [ ]:
exp = 'abrupt-4xCO2'
#model = 'MRI-ESM2-0'
#member = 'r1i1p1f1'
response_model = 'expandosc'
response_function = expandosc_function

max_t = 150
filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]

for model in member_dict:

    for member in member_dict[model]:
        result_file = model + '_' + exp + '_' + member + '_' + response_model + '_results.sav'
        
        if result_file in filenames:
            model_result = load_modelresult('../model_results/' + result_file, funcdefs = {response_model + '_function': response_function})
            par_values = model_result.best_values
            fig, axes = plt.subplots(ncols = 2, figsize = [10,4]);

            ax = axes[0]
            ax.plot(exp_part1_fixedpar(np.arange(max_t), par_values)[:max_t], color = 'lightblue')
            ax.plot(exp_part2_fixedpar(np.arange(max_t), par_values)[:max_t], color = 'lightblue')
            ax.plot(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)[:max_t], color = 'blue')
            ax.plot(osc_part1_fixedpar(np.arange(max_t), par_values)[:max_t], color = 'pink')
            ax.plot(osc_part2_fixedpar(np.arange(max_t), par_values)[:max_t], color = 'pink')
            ax.plot(osc_parts_fixedpar(np.arange(max_t), par_values)[:max_t], color = 'red')
            ax.set_title(model + ' ' + member)
            ax = axes[1]
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(exp_part1_fixedpar(np.arange(max_t), par_values)), mode = 'full')[:max_t], color = 'lightblue')
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(exp_part2_fixedpar(np.arange(max_t), par_values)), mode = 'full')[:max_t], color = 'lightblue') 
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = 'blue')
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(osc_part1_fixedpar(np.arange(max_t), par_values)), mode = 'full')[:max_t], color = 'pink')
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(osc_part2_fixedpar(np.arange(max_t), par_values)), mode = 'full')[:max_t], color = 'pink')
            ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(osc_parts_fixedpar(np.arange(max_t), par_values)), mode = 'full')[:max_t], color = 'red')
            #ax.set_title('Response to forcing increasing linearly to year 140')
            ax.grid()

## Compare 2- 3- and 4-box model (with oscillations) for abrupt-4xCO2 responses, and responses to linearly increasing forcing

In [ ]:
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
    
max_t = 150
#filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]


#for model in member_dict:
for model in ['TaiESM1', 'GISS-E2-1-G']: # some models with plateaus in abrupt-4xCO2 response
    for member in member_dict[model]:
        
        data = load_anom(model, exp, member, length_restriction = 150)
        deltaT = data['tas']; years = np.arange(1,150+1)
        if len(deltaT)<150:
            continue
        deltaT0 = np.concatenate(([0],deltaT)); years0 = np.concatenate(([0],years))

        fig, axes = plt.subplots(ncols = 2, figsize = [10,4]);

        ax = axes[0]
        ax.plot(years0, deltaT0, color = 'black')
        for i in range(len(response_models)):
            response_model = response_models[i]
            response_function = response_functions[i]
            result_file = model + '_' + exp + '_' + member + '_' + response_model + '_results.sav'

            #if result_file in filenames:
            model_result = load_modelresult('../model_results/' + result_file, funcdefs = {response_model + '_function': response_function})
            
            ax = axes[0]
            ax.plot(years0, model_result.best_fit)
            par_values = model_result.best_values

            ax.set_title(model + ' ' + member)
            ax = axes[1]
            if response_model == 'twoexp':
                ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], linewidth = 0.5)
            elif response_model == 'threeexp':
                ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t])
            elif response_model == 'expandosc':
                ax.plot(np.convolve(linear_to140years(max_t = max_t), np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], linestyle = '--', color = 'red')
           #ax.set_title('Response to forcing increasing linearly to year 140')
            ax.grid()

In [22]:
# load files downloaded from https://github.com/chrisroadmap/ar6/tree/main/data_output/SSPs
erf_ssp126_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp126_1750-2500.csv')
erf_ssp245_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp245_1750-2500.csv')
erf_ssp370_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp370_1750-2500.csv')
erf_ssp585_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp585_1750-2500.csv')
erf_years = erf_ssp126_all['year']
erf_ssp126 = erf_ssp126_all['total']
erf_ssp245 = erf_ssp245_all['total']
erf_ssp370 = erf_ssp370_all['total']
erf_ssp585 = erf_ssp585_all['total']
F4x = 7.799078433107184 # model mean 4xCO2 ERF copied from: https://github.com/chrisroadmap/ar6/blob/main/data_output/cmip6_twolayer_tuning_params.json


In [ ]:
fig, ax = plt.subplots(figsize = [12,6]);
ax.plot(erf_years, erf_ssp126)
ax.plot(erf_years, erf_ssp245)
ax.plot(erf_years, erf_ssp370)
ax.plot(erf_years, erf_ssp585)

## CMIP6 responses to SSP scenario forcing

In [ ]:
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
colors = ['green', 'red', 'blue']

max_t1 = 150    
max_t = len(erf_years)
#filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]


for model in member_dict:
#for model in ['TaiESM1', 'GISS-E2-1-G']:
    for member in member_dict[model]:
        
        data = load_anom(model, exp, member, length_restriction = 150)
        deltaT = data['tas']; years = np.arange(1,150+1)
        if len(deltaT)<150:
            continue
        deltaT0 = np.concatenate(([0],deltaT)); years0 = np.concatenate(([0],years))

        fig, axes = plt.subplots(ncols = 2, figsize = [10,4]);
        for ax in axes:
            ax.set_xlabel('Year')
            ax.set_ylabel('T [K]')
            ax.grid()

        ax = axes[0]
        ax.plot(years0, deltaT0, color = 'black')
        for i in range(len(response_models)):
            response_model = response_models[i]
            response_function = response_functions[i]
            result_file = model + '_' + exp + '_' + member + '_' + response_model + '_results.sav'

            #if result_file in filenames:
            model_result = load_modelresult('../model_results/' + result_file, funcdefs = {response_model + '_function': response_function})
            
            ax = axes[0]
            ax.plot(years0, model_result.best_fit, color = colors[i])
            par_values = model_result.best_values
            if response_model == 'expandosc':
                ax.plot(exp_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
                ax.plot(exp_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
                ax.plot(osc_parts_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
                #ax.plot(osc_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
                #ax.plot(osc_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
            

            ax.set_title(model + ' ' + member + ' abrupt-4xCO2')
            ax = axes[1]
            if response_model == 'twoexp':
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp126, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 'two-box')
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp245, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp370, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp585, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
            elif response_model == 'threeexp':
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp126, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 
                'three-box')
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp245, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp370, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp585, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
            elif response_model == 'expandosc':
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp126, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 'four-box with oscillations')
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp245, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp370, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
                ax.plot(erf_years, 1/F4x*np.convolve(erf_ssp585, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
            ax.set_title('Responses to AR6 SSP forcing')
            ax.legend(loc = 'upper left')
            #ax.set_ylim(-1,2)
            #ax.set_xlim(1750, 2100)

# LongRunMIP predictions

In [93]:
def find_longrunmip_files(model, exp):
    directory = '../../Other_data/longrunmip_data/'
    file_str = model + '_' + exp
    filenames = [f.name for f in os.scandir(directory) if file_str in f.name]
    filenames.sort()
    tas_file = filenames[1]; nettoa_file = filenames[0]
    return [tas_file, nettoa_file]

def get_tas(model, exp, add_0 = True, remove_nan = False, return_years = False):
    directory = '../../Other_data/longrunmip_data/'
    [tas_file, nettoa_file] = find_longrunmip_files(model, exp)
    ds_tas = xr.open_dataset(directory + tas_file)
    deltaT = ds_tas.tas.values
    if remove_nan == True:
        deltaT = deltaT[np.isnan(deltaT)==False]
    years = np.arange(1,len(deltaT)+1)
    if add_0 == True:
        deltaT = np.concatenate([[0],deltaT])
        years = np.concatenate(([0],years))
        
    if return_years == True:
        return [years, deltaT]
    else:
        return deltaT

In [ ]:
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
colors = ['green', 'red', 'blue']

max_t1 = 2500    
max_t = len(erf_years)
#filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]

model = 'CESM104'
exp = 'abrupt2x'
        
[years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'

fig, axes = plt.subplots(ncols = 2, figsize = [10,4]);

ax = axes[0]
ax.plot(years0, deltaT0, color = 'black')
for i in range(len(response_models)):
    response_model = response_models[i]
    response_function = response_functions[i]
    result_file = model + '_' + exp + '_' + response_model + '_results.sav'

    #if result_file in filenames:
    model_result = load_modelresult('../model_results_longrunmip/' + result_file, funcdefs = {response_model + '_function': response_function})
    
    ax = axes[0]
    ax.plot(years0, model_result.best_fit, color = colors[i])
    par_values = model_result.best_values
    if response_model == 'expandosc':
        ax.plot(exp_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(exp_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(osc_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(osc_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
    

    ax.set_title(model + ' ' + exp)
    ax = axes[1]
    if response_model == 'twoexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'threeexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'expandosc':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    #ax.set_title('Response to forcing increasing linearly to year 140')
    ax.grid()
    #ax.set_ylim(-1,4)
    #ax.set_xlim(1750, 2025)

## Zoom in

In [ ]:
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
colors = ['green', 'red', 'blue']

max_t1 = 2500   
max_t = len(erf_years)
#filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]

model = 'CESM104'
exp = 'abrupt2x'
        
[years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'

fig, axes = plt.subplots(ncols = 2, figsize = [10,4]);

ax = axes[0]
ax.plot(years0, deltaT0, color = 'black')
for i in range(len(response_models)):
    response_model = response_models[i]
    response_function = response_functions[i]
    result_file = model + '_' + exp + '_' + response_model + '_results.sav'

    #if result_file in filenames:
    model_result = load_modelresult('../model_results_longrunmip/' + result_file, funcdefs = {response_model + '_function': response_function})
    
    ax = axes[0]
    ax.plot(years0, model_result.best_fit, color = colors[i])
    par_values = model_result.best_values
    if response_model == 'expandosc':
        ax.plot(exp_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(exp_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(osc_part1_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
        ax.plot(osc_part2_fixedpar(np.arange(max_t1), par_values)[:max_t1], color = 'lightblue')
    
    ax.set_xlim(0,150)

    ax.set_title(model + ' ' + exp)
    ax = axes[1]
    if response_model == 'twoexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'threeexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'expandosc':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    #ax.set_title('Response to forcing increasing linearly to year 140')
    ax.grid()
    ax.set_ylim(-0.8,1.5)
    ax.set_xlim(1750, 2025)

The larger red spikes seems to come from the quicker time scale included for the three-box compared to the other fits.
Around 3 yrs, compared to around 6 years for other fits. Differences as a direct result of the oscillation are barely visible.